In [20]:
# dynamic predict
# collect 15 second file - 
import pyaudio
import wave

def record_audio(filename, duration=15, rate=48000, chunk=1024, channels=2, format=pyaudio.paInt16):
    audio = pyaudio.PyAudio()
    stream = audio.open(format=format, channels=channels,
                        rate=rate, input=True,
                        frames_per_buffer=chunk)
    print("Recording...")
    frames = []
    for i in range(0, int(rate / chunk * duration)):
        data = stream.read(chunk)
        frames.append(data)
    print("Finished recording.")
    
    stream.stop_stream()
    stream.close()
    audio.terminate()
    
    with wave.open(filename, 'wb') as wf:
        wf.setnchannels(channels)
        wf.setsampwidth(audio.get_sample_size(format))
        wf.setframerate(rate)
        wf.writeframes(b''.join(frames))

if __name__ == "__main__":
    filename = "recorded_audio.wav"
    record_audio(filename)

Recording...
Finished recording.


In [22]:
# break clip into "DynamicTesting/"
from pydub import AudioSegment
import os

def split_audio(filename, output_folder, segment_length=1000):
    audio = AudioSegment.from_wav(filename)
    
    if not os.path.exists(output_folder):
        os.makedirs(output_folder)
    
    for i, start_time in enumerate(range(0, len(audio), segment_length)):
        end_time = start_time + segment_length
        segment = audio[start_time:end_time]
        segment.export(os.path.join(output_folder, f"clip_{i+1}.wav"), format="wav")

if __name__ == "__main__":
    filename = "recorded_audio.wav"  # Replace with your recorded audio file
    output_folder = "DynamicTesting/Unknown"
    split_audio(filename, output_folder)

In [24]:
# preprocess - 
import os
import numpy as np
import tensorflow as tf
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import StandardScaler
import librosa.display

# Set the parent directory for speaker folders
parent_dir = "DynamicTesting"

# List of speaker folders
speaker_folders = [
    "Unknown"
]

def extract_features(parent_dir, speaker_folders):
    features = []
    labels = []

    for i, speaker_folder in enumerate(speaker_folders):
        speaker_folder_path = os.path.join(parent_dir, speaker_folder)

        for filename in os.listdir(speaker_folder_path):
            if filename.endswith(".wav"):
                file_path = os.path.join(speaker_folder_path, filename)
                audio, sr = librosa.load(file_path, sr=48000, duration=1)
                mfccs = librosa.feature.mfcc(y=audio, sr=sr, n_mfcc=90)
                
                # Normalize MFCC features
                mfccs = StandardScaler().fit_transform(mfccs)
                
                features.append(mfccs.T)
                labels.append(i)

    return np.array(features), np.array(labels)

# Extract features and labels
X, y = extract_features(parent_dir, speaker_folders)

In [28]:
import os
import numpy as np
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import accuracy_score, f1_score
import joblib


# Initialize a dictionary to store predictions and counts for each model
predictions_counts = {}

# Iterate over each model in the models folder
models_folder = "models"
for model_file in os.listdir(models_folder):
    if model_file.endswith(".joblib"):
        name = model_file.split("VsNon")[0]
        user_speaking = name
        non_user_speaking = "Non" + name
        actual_speakers = [user_speaking,non_user_speaking]

        label_encoder = LabelEncoder()
        label_encoder.classes_ = np.array(actual_speakers)

        # Load the model
        model = joblib.load(os.path.join(models_folder, model_file))
        
        # Make predictions
        y_pred_probabilities = model.predict(X)
        y_pred = np.argmax(y_pred_probabilities, axis=1)
        y_pred_decoded = label_encoder.inverse_transform(y_pred)
        
        # Count the occurrences of each unique element
        unique_elements, counts = np.unique(y_pred_decoded, return_counts=True)
        predictions_counts[model_file] = dict(zip(unique_elements, counts))
        
        # Print the predictions
        print(f"Predictions for model {model_file}:")
        print(y_pred_decoded)
        print("Counts of each unique element:")
        print(predictions_counts[model_file])
        print("\n")

# You can access the predictions and counts for each model from the predictions_counts dictionary

1/1 [==============================] - 1s 534ms/step
Predictions for model AbhishekVsNonAbhishek.joblib:
['NonAbhishek' 'NonAbhishek' 'NonAbhishek' 'NonAbhishek' 'NonAbhishek'
 'NonAbhishek' 'NonAbhishek' 'NonAbhishek' 'NonAbhishek' 'NonAbhishek'
 'NonAbhishek' 'NonAbhishek' 'NonAbhishek' 'NonAbhishek' 'NonAbhishek']
Counts of each unique element:
{'NonAbhishek': 15}


1/1 [==============================] - 1s 500ms/step
Predictions for model AnirbanVsNonAnirban.joblib:
['NonAnirban' 'NonAnirban' 'NonAnirban' 'NonAnirban' 'NonAnirban'
 'NonAnirban' 'NonAnirban' 'NonAnirban' 'NonAnirban' 'NonAnirban'
 'NonAnirban' 'NonAnirban' 'NonAnirban' 'NonAnirban' 'NonAnirban']
Counts of each unique element:
{'NonAnirban': 15}


1/1 [==============================] - 0s 500ms/step
Predictions for model ArunVsNonArun.joblib:
['NonArun' 'NonArun' 'NonArun' 'NonArun' 'NonArun' 'NonArun' 'NonArun'
 'NonArun' 'NonArun' 'NonArun' 'NonArun' 'NonArun' 'NonArun' 'NonArun'
 'NonArun']
Counts of each unique e